In [ ]:
# use nba api to get league data from nba 
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import playercareerstats
import time

In [ ]:
# download the mvp voting dataset from kaggle
import kagglehub

path = kagglehub.dataset_download("parthdande/nba-mvp-voting-dataset-2000-2021")
print("Path to dataset files:", path)

Path to dataset files: C:\Users\alyre\.cache\kagglehub\datasets\parthdande\nba-mvp-voting-dataset-2000-2021\versions\3


In [ ]:
# move the csv files of the mvp voting into the current data directory
import shutil
import os

destination_folder = "./data"
if not os.path.exists(destination_folder):
    os.makedirs(destination_folder)

for filename in os.listdir(path):
    source_file = os.path.join(path, filename)
    destination_file = os.path.join(destination_folder, filename)
    
    shutil.copy(source_file, destination_file)
    print(f"Successfully moved: {filename} to {destination_folder}")


Successfully moved: 2001-2010 MVP Data.csv to ./data
Successfully moved: 2010-2021 MVP Data.csv to ./data
Successfully moved: 2022-2023 MVP Data.csv to ./data
Successfully moved: PlayerStats_2023-24.csv to ./data


In [ ]:
# function to get season stats

def get_season_stats(curr_season):
    stats = leaguedashplayerstats.LeagueDashPlayerStats(season=curr_season)
    df = stats.get_data_frames()[0]
    df['Season'] = curr_season
    return df

In [ ]:
# get the data from the seasons

all_seasons_data = []
seasons = ['2020-21', '2021-22', '2022-23']

for year in seasons:
    df = get_season_stats(year)
    all_seasons_data.append(df)
    time.sleep(2)

In [23]:
print(all_seasons_data[0]["Season"])

0      2020-21
1      2020-21
2      2020-21
3      2020-21
4      2020-21
        ...   
535    2020-21
536    2020-21
537    2020-21
538    2020-21
539    2020-21
Name: Season, Length: 540, dtype: object
